In [1]:
Set([1, 2,3])

Set{Int64} with 3 elements:
  2
  3
  1

PyRes

unification

Norvig unify
https://github.com/aimacode/aima-python/blob/9ea91c1d3a644fdb007e8dd0870202dcd9d078b6/logic4e.py#L1307

norvig - widespread error
http://norvig.com/unify-bug.pdf

Efficient unification
ftp://ftp.cs.indiana.edu/pub/techreports/TR242.pdf

blog post
https://eli.thegreenplace.net/2018/unification/

Efficient representations for triangular substitituions
https://users.soe.ucsc.edu/~lkuper/papers/walk.pdf

conor mcbride - first order substitition structurly recursive dependent types
http://citeseerx.ist.psu.edu/viewdoc/download;jsessionid=880725E316FA5E3540EFAD83C0C2FD88?doi=10.1.1.25.1516&rep=rep1&type=pdf

z3 unifier
https://github.com/Z3Prover/z3/blob/520ce9a5ee6079651580b6d83bc2db0f342b8a20/src/ast/substitution/unifier.cpp



Schulz 2009 tutorial
https://resources.mpi-inf.mpg.de/departments/rg1/conferences/vtsa09/slides/schulz.pdf



A disjoint set data struture in julia. Doesn't look crzy
https://github.com/JuliaCollections/DataStructures.jl/blob/master/src/disjoint_set.jl



In [8]:
#--------------------
abstract type Symbolic{T} end

#variables
struct Sym{T} <: Symbolic{T}
    name::Symbol
end

struct Term{T} <: Symbolic{T}
    f::Any
    arguments::Ar
end



In [11]:
#variables
struct Sym
    name::Symbol
end

struct Term
    f::Symbol
    arguments::Array{Any} # Array{Union{Term,Sym}}
end

In [10]:
Sym{Int64}(:x)

Sym{Int64}(:x)

In [11]:
any( [true true false]  )

true

In [21]:
size([1 2 3])

(1, 3)

In [12]:
occur_check(x::Sym,y::Term,s) = any(occur_check(x, a, s) for a in y.arguments)

function occur_check(x::Sym,y::Sym,s)
    if x == y
        return s
    elseif haskey(s,y)
        return occur_check(x, s[y], s)
    else
        return nothing
    end  
end


function unify(x::Sym, y::Union{Sym,Term}, s) 
   if x == y
        return s
   elseif haskey(s,x)
        return unify(s[x], y, s)
   elseif haskey(s,y) # This is the norvig twist
        return unify(x, s[y], s)
   elseif occur_check(x,y,s)
        return nothing
   else
        s[x] = y
        return s
   end
end

unify(x::Term, y::Sym, s) = unify(y,x,s)

function unify(x :: Term, y :: Term, s)
    if x.f == y.f && length(x.arguments) == length(y.arguments)
        for (x1, y1) in zip(x.arguments, y.arguments)
            if unify(x1,y1,s) == nothing
                return nothing
            end
        end
        return s
    else
        return nothing
    end
end

unify(x,y) = unify(x,y,Dict())

unify (generic function with 4 methods)

In [16]:
s = Dict()
s[3] = 2
s[4] = 4
s

Dict{Any,Any} with 2 entries:
  4 => 4
  3 => 2

In [13]:
#==
string2term(x)
    if x isa Symbol
        name = String(x)
        if isuppercase(name[1])
           return :( Sym( Symbol($(name)) ) )
        else
           return :( Term( Symbol($(name)), []  ))
        end
    elseif x isa Expr
        @assert(x.head == :call)
        arguments = [string2term(y) for y in x.args[2:end] ]
        return :( Term(  Symbol($(String(x.args[1]))), $(arguments)  ))
    end
end
==#
function string2term(x)
    if x isa Symbol
        name = String(x)
        if isuppercase(name[1])
           return Sym( x)
        else
           return Term( x, []  )
        end
    elseif x isa Expr
        @assert(x.head == :call)
        arguments = [string2term(y) for y in x.args[2:end] ]
        return Term( x.args[1], arguments )
    end
end
macro string2term(x)
    return :( $(string2term(x)) )
end


@string2term (macro with 1 method)

In [3]:
string2term(:(f(x)))

f(x)
x


Term(:f, Any[Term(:x, Any[])])

In [6]:
@string2term(f(y,Yy,Z))

f(y, Yy, Z)
y
Yy
Z


Term(:f, Any[Term(:y, Any[]), Sym(:Yy), Sym(:Z)])

In [9]:
s = Dict()
unify( @string2term(X) , @string2term(a) , s)
s

X
a


Dict{Any,Any} with 1 entry:
  Sym(:X) => Term(:a, Any[])

In [15]:
s = Dict()
unify( @string2term(X) , @string2term(f(X)) , s)


X
f(X)
X


false

In [17]:
s = Dict()
unify( @string2term(X) , @string2term(f(Y)) , s)
s

X
f(Y)
Y


Dict{Any,Any} with 1 entry:
  Sym(:X) => Term(:f, Any[Sym(:Y)])

In [23]:
s = Dict()
unify( @string2term(f(X,a)) , @string2term(f(b,Y)) , s)
s

f(X, a)
X
a
f(b, Y)
b
Y


Dict{Any,Any} with 2 entries:
  Sym(:X) => Term(:b, Any[])
  Sym(:Y) => Term(:a, Any[])

In [14]:

print(unify( @string2term(p(X,g(a), f(a, f(a)))) , @string2term(p(f(a), g(Y), f(Y, Z)))))


Dict{Any,Any}(Sym(:X) => Term(:f, Any[Term(:a, Any[])]),Sym(:Y) => Term(:a, Any[]),Sym(:Z) => Term(:f, Any[Term(:a, Any[])]))

In [35]:
s = Dict()
print(unify( @string2term(f(f(g(X),a),g(X))) , @string2term(f(f(g(X),a),g(X))) , s))
s

f(f(g(X), a), g(X))
f(g(X), a)
g(X)
X
a
g(X)
X
f(f(g(X), a), g(X))
f(g(X), a)
g(X)
X
a
g(X)
X
true

Dict{Any,Any} with 0 entries

In [40]:
unify( @string2term(p(X,Y)) , @string2term(p(Y,X)) , s)


Dict{Any,Any} with 1 entry:
  Sym(:X) => Sym(:Y)

In [ ]:
        self.s9 = terms.string2Term("f(f(g(X),a),X)")
        self.t9 = terms.string2Term("f(Y,g(Y))")

        self.s10 = terms.string2Term("f(f(g(X),a),g(X))")
        self.t10 = terms.string2Term("f(Y,g(Z))")

        self.s11 = terms.string2Term("p(X,g(a), f(a, f(a)))")
        self.t11 = terms.string2Term("p(f(a), g(Y), f(Y, Z))")

In [52]:
Symbol("x")

:x

In [27]:
isuppercase(String(:x)[1])

false

In [5]:
s = Dict{Sym, Union{Sym , Term}}()
s[Sym(:x)] = Sym(:y)
s

Dict{Sym,Union{Sym, Term}} with 1 entry:
  Sym(:x) => Sym(:y)

In [13]:
Term(:f, []) in s

ErrorException: AbstractDict collections only contain Pairs;
Either look for e.g. A=>B instead, or use the `keys` or `values`
function if you are looking for a key or value respectively.

In [15]:
haskey(s, Term(:f, []))

false

In [17]:
haskey(s, Sym(:x))

true

In [8]:
typeof(:( 1 + 2 ))

Expr

In [9]:
typeof(:x)

Symbol

In [12]:
:(1 + 2).head

:call

In [ ]:
struct ProofState
 processed
 unprocessed
end


struct Clause
    
end

In [ ]:
computeAllFactors

function computeAllResolvents(clauses , processed ) 
end

function computeAllFactors

SAT solving


https://github.com/niklasso/minisat/blob/master/minisat/core/Solver.cc


nelson oppen




One should always consider brute force first


Davis Putnam solver via reslutions


First order axioms of categroy theory
https://math.stackexchange.com/questions/2383503/category-theory-from-the-first-order-logic-point-of-view#:~:text=Any%20model%20of%20this%20theory%20is%20a%20category.&text=First%2Dorder%20logic%20is%20typically,%22%20or%20%22internal%22%20categories.

https://www.cs.le.ac.uk/people/rlc3/research/papers/mgs2015-categoryTheory-exercises.pdf

Category theory questions
pullbacks of monics are monic

verify x is a category
verify x is a functor



In [ ]:
struct Solver1
    clauses::Array{Array{Int64}}
    
end

nvars(s::Solver) = maximum(abs.(s.clauses))

function is_sat( state, clauses )
    all( any( state[lit] for lit in clause)   for clause in clauses )
end
    



function search1(s, state, n)
    if n >= nvars
        return is_sat(state, s.clauses)
    end
    
    state[n] = true
    if search(s, state, n+1 )
      return true
    end
    
    state[n] = false
    if search(s, state, n+1 )
      return true
    end
    
    return false
end


# iterative version
function search2(s)
    state = [true for i in 0:nvars(s)]
    i = n
    fw = false
    while i >= 1
        if i == n && is_sat(state)
             break
        end
        if fw
            state[i] = true
            i += 1
        elseif !fw && state[i]
            state[i] = false
            fw = true
            i += 1
        elseif !fw && !state[i]
            state[i] = true
            i -= 1
        end
        
    end
end



function search3()
    stack = [picklit]
    state = []
    while length(stack) >= 0 
        if alldone
            is_sat(s,state)
        end
        n = picklit
        if state[stack[end]] # push onto stack
            stack.push(picklit(s))
        else
            state[]
        end
        state[i] = true
        stack.push(n)
    end


end



In [47]:
maximum(abs.([[1 2] [-3] [4]]))

4

In [55]:
maximum(map(x -> maximum(abs.(x)) ,  [[1,2], [-3]]))

3

In [9]:
function foo(n)
    acc = n
    for i in 1:n
        acc += i*acc
    end
    return acc
end

@code_llvm(foo(7))


;  @ In[9]:2 within `foo'
define i64 @julia_foo_17600(i64) {
top:
;  @ In[9]:3 within `foo'
; ┌ @ range.jl:5 within `Colon'
; │┌ @ range.jl:277 within `UnitRange'
; ││┌ @ range.jl:282 within `unitrange_last'
; │││┌ @ operators.jl:341 within `>='
; ││││┌ @ int.jl:410 within `<='
       %1 = icmp sgt i64 %0, 0
; │││└└
     %2 = select i1 %1, i64 %0, i64 0
; └└└
  br i1 %1, label %L12, label %L28

L12:                                              ; preds = %top, %L12
  %value_phi3 = phi i64 [ %4, %L12 ], [ %0, %top ]
  %value_phi4 = phi i64 [ %6, %L12 ], [ 1, %top ]
;  @ In[9]:4 within `foo'
; ┌ @ int.jl:54 within `*'
   %3 = mul i64 %value_phi4, %value_phi3
; └
; ┌ @ int.jl:53 within `+'
   %4 = add i64 %3, %value_phi3
; └
; ┌ @ range.jl:597 within `iterate'
; │┌ @ promotion.jl:398 within `=='
    %5 = icmp eq i64 %value_phi4, %2
; │└
   %6 = add nuw i64 %value_phi4, 1
; └
  br i1 %5, label %L28, label %L12

L28:                                              ; preds = %L12, %top
  %value